In [2]:
!pip install opendatasets

  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73053 sha256=13dd37300a5c46c5ceb7d1ea514e60f8759ebd65463ea887d0f4523f518f3f4f
  Stored in directory: c:\users\donat\appdata\local\pip\cache\wheels\29\da\11\144cc25aebdaeb4931b231e25fd34b394e6a5725cbb2f50106
Successfully built kaggle


In [3]:
!pip install tensorflow

  Created wheel for clang: filename=clang-5.0-py3-none-any.whl size=30702 sha256=e89aa74750e1c100124ec380fca3ec922eefe1b444907467b6050a3969ab1e0f
  Stored in directory: c:\users\donat\appdata\local\pip\cache\wheels\f1\60\77\22b9b5887bd47801796a856f47650d9789c74dc3161a26d608
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4829 sha256=b1dbfd20f0298628d3acb611bbb8783bdd48f7f130d4ff9336e44f0455dee328
  Stored in directory: c:\users\donat\appdata\local\pip\cache\wheels\a0\16\9c\5473df82468f958445479c59e784896fa24f4a5fc024b0f501
Successfully built clang termcolor
  Attempting uninstall: numpy
    Found existing installation: numpy 1.20.1
    Uninstalling numpy-1.20.1:
      Successfully uninstalled numpy-1.20.1
  Attempting uninstall: h5py
    Found existing installation: h5py 2.10.0
    Uninstalling h5py-2.10.0:
      Successfully uninstalled h5py-2.10.0


In [4]:
!pip install tensorflow_federated

  Created wheel for absl-py: filename=absl_py-0.9.0-py3-none-any.whl size=121930 sha256=dc16daa7c4e7aedf58e7fac736dd3c327c65cc57c4f0e7d892565f5127646575
  Stored in directory: c:\users\donat\appdata\local\pip\cache\wheels\1d\10\8e\2f79b924179ff1e6510933d63eb851bea01054fff262343b7a
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11429 sha256=165cffd8d70b855439cae8b753d1b5431cf1f23e6003b655bc794ccdf66c4d74
  Stored in directory: c:\users\donat\appdata\local\pip\cache\wheels\c4\a7\48\0a434133f6d56e878ca511c0e6c38326907c0792f67b476e56
Successfully built absl-py retrying
  Attempting uninstall: cachetools
    Found existing installation: cachetools 4.2.2
    Uninstalling cachetools-4.2.2:
      Successfully uninstalled cachetools-4.2.2
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.40.0
  

In [18]:
!pip install torch

In [19]:
!pip install torchvision

In [1]:
import opendatasets as od
from tensorflow import keras
import numpy as np

In [2]:
od.download("https://www.kaggle.com/prashant268/chest-xray-covid19-pneumonia")
od.download("https://www.kaggle.com/sudalairajkumar/novel-corona-virus-2019-dataset")

Skipping, found downloaded files in ".\chest-xray-covid19-pneumonia" (use force=True to force download)
Skipping, found downloaded files in ".\novel-corona-virus-2019-dataset" (use force=True to force download)


In [3]:
LEARNING_RATE = 0.001 # 0.0001
MAX_EPOCHS = 25
TARGET_FOLDER = "weights"

In [4]:
import torch
import torchvision
from torchvision import datasets, transforms

Resize Images to 244, 244. By using to_tensor the images are already normalized between 0 and 1. "The image object is an array of (244, 244, 3) should be flattened to be list (178, 608)." What? wie?

In [5]:
transform = transforms.Compose([transforms.Resize((244, 244))
                                , transforms.ToTensor()]
                               #, transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # find mean and std of dataset
                              )

test_set = datasets.ImageFolder('chest-xray-covid19-pneumonia/Data/test', transform=transform)

train_set = dataset = datasets.ImageFolder('chest-xray-covid19-pneumonia/Data/train', transform=transform)

In [6]:
def label_preparation(labels):
    labels = np.array(labels)
    labels[labels > 0] = 1
    return list(labels)

def label_preparation_tensor(labels):
    labels[labels > 0] = 1
    return labels

train_set.targets = label_preparation(train_set.targets)

test_set.targets = label_preparation(test_set.targets)

In [7]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True, num_workers=4)

test_loader = torch.utils.data.DataLoader(test_set, batch_size=64, shuffle=False, num_workers=4)

In [8]:
train_loader = None
test_loader = None

In Paper aus References haben sie unter anderen ResNet18 benutzt: https://arxiv.org/pdf/2007.05592.pdf

Deshalb würde ich das probieren.

TODO: test different ones, e.g. VGG19

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(f"Current device: {device}")

Current device: cuda:0


In [10]:
def calc_accuracy(result, labels):
    result = torch.sigmoid(result).round()
    
    correct_results_sum = (result == labels).sum().float()
    acc = correct_results_sum/labels.shape[0]
    acc *= 100
    
    return acc


In [11]:
def train(model, data_loader, optimizer, loss):
    """
    model -- neural net
    data_loader -- dataloader for train images
    optimizer -- optimizer
    """
    model.train()
    
    # use pos weights because of unbalanced data set
    accuracy = 0
    for step, [images, labels] in enumerate(data_loader, 1):
        images = images.to(device)
        labels = label_preparation_tensor(labels.to(device))

        optimizer.zero_grad()
        
        result = model(images)
        targets = labels.unsqueeze(1).float()

        loss_value = loss(result.float(), targets)

        # backpropagation
        loss_value.backward()
        optimizer.step()
                                    
        if step % 10 == 0:
            accuracy += calc_accuracy(result, labels.unsqueeze(1))
            print(f"TRAINING - Step: {step}, loss: {loss_value}, rolling accuracy: {accuracy*10/step}")

In [12]:
def test(model, test_loader, loss):
    """    
    model -- neural net 
    test_loader -- dataloader of test images
    epoch -- current epoch
    """
    model.eval()
    
    with torch.no_grad():
        loss_value = 0
        accuracy = 0
        for step, [images, labels] in enumerate(test_loader, 1):
            images = images.to(device)
            labels = label_preparation_tensor(labels.to(device))

            result = model(images)
            targets = labels.detach().unsqueeze(1).float()

            loss_value += loss(result.detach(), targets)
            accuracy += calc_accuracy(result.detach(), labels.detach().unsqueeze(1))

        loss_value /= step
        accuracy /=  step
  
    print(f"TESTING - Loss: {loss_value}, Accuracy: {accuracy}")
    return accuracy > 93.

In [13]:
def run_local_training():
    model = torchvision.models.resnet18(pretrained=False, num_classes=1)
    model.to(device)

    # initialize optimizer
    optimizer = torch.optim.Adam(model.parameters())

    # use pos weights because of unbalanced data set
    loss = torch.nn.BCEWithLogitsLoss(pos_weight=torch.tensor([1./10])).to(device) # binary crossentropy

    # start training
    for epoch in range(MAX_EPOCHS):
        print(f"+++ EPOCH: {epoch+1} +++++++++")
        torch.nn.BCEWithLogitsLoss(pos_weight=torch.tensor([1./10])).to(device) # binary crossentropy (non-federated)
        train(model, train_loader, optimizer, loss)
        break
        # save interim weights
        torch.save(model.state_dict(), f'./{TARGET_FOLDER}/epoch_{epoch}.ckpt')

        if test(model, test_loader, loss) and epoch > 4:
            break

In [14]:
#run_local_training()

# Federated
https://blog.openmined.org/upgrade-to-federated-learning-in-10-lines/

i don't think we need syft if we just simulate the federation on one machine

In [15]:
from collections import OrderedDict

In [16]:
number_of_clients = 4

In [17]:
federated_model = torchvision.models.resnet18(pretrained=False, num_classes=1)
federated_model.to(device)

def reset_client_param_list():
    return [None] * number_of_clients
    
client_params = reset_client_param_list()

In [18]:
client_models = [torchvision.models.resnet18(pretrained=False, num_classes=1).to(device) for _ in range(number_of_clients)]
client_optimizers = [torch.optim.Adam(client_models[idx].parameters()) for idx in range(number_of_clients)]
client_training_loader = [torch.utils.data.DataLoader(train_set, batch_size=8, shuffle=True, num_workers=4) for _ in range(number_of_clients)]

federated_test_loader = torch.utils.data.DataLoader(test_set, batch_size=8, shuffle=False, num_workers=4)

In [19]:
def send_params_to_global_model(client_params, client_idx, params):
    client_params[client_idx] = params
    for idx in range (number_of_clients):
        if client_params[idx] is None:
            return client_params, False
        
    return client_params, True

def send_params_to_client(federated_model, client):
    client.load_state_dict(federated_model.state_dict(), True)
    return client
    
def federated_average(federated_model, client_params):
    with torch.no_grad():
        average_weights = OrderedDict()

        # check if client_params has no Nones?
        for client_param in client_params:
            for key, value in client_param.items():
                if key in average_weights:
                    average_weights[key] += value.clone()
                else:
                    average_weights[key] = value.clone()

        for key, value in average_weights.items():
            average_weights[key] = torch.div(average_weights[key], len(client_params), rounding_mode='floor')
                
                
    federated_model.load_state_dict(average_weights, True)
    return federated_model
    

In [20]:
def train_federated(model, data_loader, optimizer, loss):
    """
    model -- neural net
    data_loader -- dataloader for train images
    optimizer -- optimizer
    """
    model.train()
    
    # use pos weights because of unbalanced data set
    accuracy = 0
    for step, [images, labels] in enumerate(data_loader, 1):
        images = images.to(device)
        labels = label_preparation_tensor(labels.to(device))

        optimizer.zero_grad()
        
        result = model(images)
        targets = labels.unsqueeze(1).float()

        loss_value = loss(result.float(), targets)

        # backpropagation
        loss_value.backward()
        optimizer.step()
                                    
        if step % 10 == 0:
            accuracy += calc_accuracy(result, labels.unsqueeze(1))
            print(f"TRAINING - Step: {step}, loss: {loss_value}, rolling accuracy: {accuracy*10/step}")

In [21]:
def test_federated(model, test_loader, loss):
    """    
    model -- neural net 
    test_loader -- dataloader of test images
    epoch -- current epoch
    """
    model.eval()
    
    # use pos weights because of unbalanced data set
    criterion = torch.nn.BCEWithLogitsLoss(pos_weight=torch.tensor([1./10])).to(device) # binary crossentropy (non-federated)
    
    with torch.no_grad():
        loss_value = 0
        accuracy = 0
        for step, [images, labels] in enumerate(test_loader, 1):
            images = images.to(device)
            labels = label_preparation_tensor(labels.to(device))

            result = model(images)
            targets = labels.detach().unsqueeze(1).float()

            loss_value += loss(result.detach(), targets)
            accuracy += calc_accuracy(result.detach(), labels.detach().unsqueeze(1))

        loss_value /= step
        accuracy /=  step
  
    print(f"TESTING - Loss: {loss_value}, Accuracy: {accuracy}")
    return accuracy > 93.

In [22]:
def run_federated_training(federated_model, client_models, client_optimizers, client_params, client_training_loader):
    # use pos weights because of unbalanced data set
    federated_loss = torch.nn.BCEWithLogitsLoss(pos_weight=torch.tensor([1./10])).to(device) # binary crossentropy
    # federated_loss = torch.nn.CrossEntropyLoss(weight=torch.tensor([1./10])).to(device) # sparce categorical crossentropy (federated)

    # start training
    for epoch in range(MAX_EPOCHS):
        for client_idx in range (number_of_clients):
            print(f"+++ FEDERATED MODEL {client_idx}, EPOCH: {epoch+1} +++++++++")

            client_model = client_models[client_idx]
            client_model = send_params_to_client(federated_model, client_model)

            client_optimizer = client_optimizers[client_idx]

            train_federated(client_model, client_training_loader[client_idx], client_optimizer, federated_loss)

            client_params, _ = send_params_to_global_model(client_params, client_idx, client_model.state_dict())

            # save interim weights
            #torch.save(model.state_dict(), f'./{TARGET_FOLDER}/epoch_{epoch}.ckpt')

        federated_model = federated_average(federated_model, client_params)
        client_params = reset_client_param_list()
        
        test_federated(federated_model, federated_test_loader, federated_loss)


In [23]:
run_federated_training(federated_model, client_models, client_optimizers, client_params, client_training_loader)

+++ FEDERATED MODEL 0, EPOCH: 1 +++++++++


C:\Users\Donat\anaconda3\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


TRAINING - Step: 10, loss: 0.1328851580619812, rolling accuracy: 75.0
TRAINING - Step: 20, loss: 0.043614745140075684, rolling accuracy: 81.25
TRAINING - Step: 30, loss: 0.1180800199508667, rolling accuracy: 79.16667175292969
TRAINING - Step: 40, loss: 0.020529018715023994, rolling accuracy: 84.375
TRAINING - Step: 50, loss: 0.5681607127189636, rolling accuracy: 85.0
TRAINING - Step: 60, loss: 0.1679622232913971, rolling accuracy: 77.08333587646484
TRAINING - Step: 70, loss: 0.06908659636974335, rolling accuracy: 75.0
TRAINING - Step: 80, loss: 0.020371772348880768, rolling accuracy: 78.125
TRAINING - Step: 90, loss: 0.061381686478853226, rolling accuracy: 79.16667175292969
TRAINING - Step: 100, loss: 0.4305272102355957, rolling accuracy: 80.0
TRAINING - Step: 110, loss: 0.4173275828361511, rolling accuracy: 78.40908813476562
TRAINING - Step: 120, loss: 0.01620090752840042, rolling accuracy: 80.20833587646484
TRAINING - Step: 130, loss: 0.015924273058772087, rolling accuracy: 81.730766

TRAINING - Step: 370, loss: 0.009386278688907623, rolling accuracy: 89.8648681640625
TRAINING - Step: 380, loss: 0.02614017389714718, rolling accuracy: 89.8026351928711
TRAINING - Step: 390, loss: 0.041117098182439804, rolling accuracy: 89.74359130859375
TRAINING - Step: 400, loss: 0.05815167725086212, rolling accuracy: 89.6875
TRAINING - Step: 410, loss: 0.07526841759681702, rolling accuracy: 89.63414764404297
TRAINING - Step: 420, loss: 0.02711445838212967, rolling accuracy: 89.58333587646484
TRAINING - Step: 430, loss: 0.018263189122080803, rolling accuracy: 89.53488159179688
TRAINING - Step: 440, loss: 0.04973895102739334, rolling accuracy: 89.48863220214844
TRAINING - Step: 450, loss: 0.01250607892870903, rolling accuracy: 89.72222137451172
TRAINING - Step: 460, loss: 0.009420952759683132, rolling accuracy: 89.94564819335938
TRAINING - Step: 470, loss: 0.018845807760953903, rolling accuracy: 90.1595687866211
TRAINING - Step: 480, loss: 0.0030417917296290398, rolling accuracy: 90.3

TRAINING - Step: 80, loss: 0.028262455016374588, rolling accuracy: 76.5625
TRAINING - Step: 90, loss: 0.0599149689078331, rolling accuracy: 79.16667175292969
TRAINING - Step: 100, loss: 0.01970897614955902, rolling accuracy: 80.0
TRAINING - Step: 110, loss: 0.101010762155056, rolling accuracy: 78.40908813476562
TRAINING - Step: 120, loss: 0.03463415428996086, rolling accuracy: 80.20833587646484
TRAINING - Step: 130, loss: 0.08371539413928986, rolling accuracy: 78.84615325927734
TRAINING - Step: 140, loss: 0.03627185523509979, rolling accuracy: 79.46428680419922
TRAINING - Step: 150, loss: 0.011906055733561516, rolling accuracy: 80.83333587646484
TRAINING - Step: 160, loss: 0.03652975335717201, rolling accuracy: 81.25
TRAINING - Step: 170, loss: 0.020044676959514618, rolling accuracy: 82.35294342041016
TRAINING - Step: 180, loss: 0.1106523871421814, rolling accuracy: 82.6388931274414
TRAINING - Step: 190, loss: 0.03141968697309494, rolling accuracy: 82.89473724365234
TRAINING - Step: 20

...

TRAINING - Step: 490, loss: 0.14750966429710388, rolling accuracy: 72.95918273925781
TRAINING - Step: 500, loss: 0.06837034225463867, rolling accuracy: 73.5
TRAINING - Step: 510, loss: 0.14746274054050446, rolling accuracy: 73.7745132446289
TRAINING - Step: 520, loss: 0.14759117364883423, rolling accuracy: 74.03845977783203
TRAINING - Step: 530, loss: 0.14744383096694946, rolling accuracy: 74.2924575805664
TRAINING - Step: 540, loss: 0.1475229263305664, rolling accuracy: 74.53703308105469
TRAINING - Step: 550, loss: 0.14794638752937317, rolling accuracy: 74.7727279663086
TRAINING - Step: 560, loss: 0.14751310646533966, rolling accuracy: 75.0
TRAINING - Step: 570, loss: 0.14789634943008423, rolling accuracy: 75.21929931640625
TRAINING - Step: 580, loss: 0.06831462681293488, rolling accuracy: 75.64655303955078
TRAINING - Step: 590, loss: 0.14761953055858612, rolling accuracy: 75.84745788574219
TRAINING - Step: 600, loss: 0.1473805010318756, rolling accuracy: 76.04167175292969
TRAINING - 

TRAINING - Step: 210, loss: 0.06326134502887726, rolling accuracy: 73.21428680419922
TRAINING - Step: 220, loss: 0.05593350902199745, rolling accuracy: 74.43181610107422
TRAINING - Step: 230, loss: 0.36827027797698975, rolling accuracy: 74.4565200805664
TRAINING - Step: 240, loss: 0.14862331748008728, rolling accuracy: 75.00000762939453
TRAINING - Step: 250, loss: 0.04954316467046738, rolling accuracy: 76.0
TRAINING - Step: 260, loss: 0.1444822996854782, rolling accuracy: 76.44230651855469
TRAINING - Step: 270, loss: 0.14565879106521606, rolling accuracy: 76.38888549804688
TRAINING - Step: 280, loss: 0.05785021185874939, rolling accuracy: 76.78571319580078
TRAINING - Step: 290, loss: 0.04486551135778427, rolling accuracy: 77.5862045288086
TRAINING - Step: 300, loss: 0.2688705325126648, rolling accuracy: 77.08333587646484
TRAINING - Step: 310, loss: 0.05311489850282669, rolling accuracy: 77.82257843017578
TRAINING - Step: 320, loss: 0.15565015375614166, rolling accuracy: 77.734375
TRAIN

TRAINING - Step: 570, loss: 0.0633307546377182, rolling accuracy: 77.19298553466797
TRAINING - Step: 580, loss: 0.2656254172325134, rolling accuracy: 77.1551742553711
TRAINING - Step: 590, loss: 0.0649385154247284, rolling accuracy: 77.54237365722656
TRAINING - Step: 600, loss: 0.07185317575931549, rolling accuracy: 77.5
TRAINING - Step: 610, loss: 0.1532563865184784, rolling accuracy: 77.66393280029297
TRAINING - Step: 620, loss: 0.0645902156829834, rolling accuracy: 78.0241928100586
TRAINING - Step: 630, loss: 0.06343637406826019, rolling accuracy: 78.17460632324219
TRAINING - Step: 640, loss: 0.0628407746553421, rolling accuracy: 78.3203125
+++ FEDERATED MODEL 3, EPOCH: 3 +++++++++
TRAINING - Step: 10, loss: 7.370444297790527, rolling accuracy: 12.5
TRAINING - Step: 20, loss: 6.51995849609375, rolling accuracy: 50.0
TRAINING - Step: 30, loss: 5.647168159484863, rolling accuracy: 54.16666793823242
TRAINING - Step: 40, loss: 5.045135974884033, rolling accuracy: 59.375
TRAINING - Step:

TRAINING - Step: 290, loss: 0.16172467172145844, rolling accuracy: 22.84482765197754
TRAINING - Step: 300, loss: 0.22463029623031616, rolling accuracy: 22.5
TRAINING - Step: 310, loss: 0.5421658158302307, rolling accuracy: 22.177419662475586
TRAINING - Step: 320, loss: 0.32869142293930054, rolling accuracy: 21.484375
TRAINING - Step: 330, loss: 0.5017850399017334, rolling accuracy: 21.59090805053711
TRAINING - Step: 340, loss: 0.260666161775589, rolling accuracy: 21.691177368164062
TRAINING - Step: 350, loss: 0.11426043510437012, rolling accuracy: 22.5
TRAINING - Step: 360, loss: 0.12502487003803253, rolling accuracy: 22.916667938232422
TRAINING - Step: 370, loss: 0.15163320302963257, rolling accuracy: 23.31081199645996
TRAINING - Step: 380, loss: 0.11800909787416458, rolling accuracy: 23.68421173095703
TRAINING - Step: 390, loss: 0.0793057233095169, rolling accuracy: 24.038461685180664
TRAINING - Step: 400, loss: 0.15283195674419403, rolling accuracy: 24.0625
TRAINING - Step: 410, los

KeyboardInterrupt: 